In [1]:
import pandas as pd

In [2]:
# Ten year model 2022 to 2032
# Revenue (year 2022) = 222
# Revenue Groth Factor 18%
# EBITDA Marging = 45% Current Revenue
# Taxes = 21% current EBITDA
# Capital Expenditure = 3% Current Revenue
# Working Capital Increase 6% Revenue
# EDBITA Exit Multiple: 32.70
# Discount Factor(WACC): 0.94 (6%).

In [3]:
initial_revenue = 222
growth_factor = 0.18
ebitda_margin = 0.45
taxes = 0.21
capital_expenditures = 0.03
working_capital_increase = 0.06
ebitda_exit_multiple = 32.70
discount_factor_rate = 0.06

In [4]:
df = pd.DataFrame(data={"Revenue": growth_factor,
                        "EBITDA": 0,
                        "Taxes": 0,
                        "Capital Expenditure": 0,
                        "Working Capital": 0,
                        "Cash Flow": 0,
                        "Discount Factor": (1 + discount_factor_rate) ** -1,
                        "Present Value of Free Cash Flow": 0,
                        },
                  index=pd.date_range(start="2022",
                                      end="2033",
                                      freq="Y",
                                      name="Date")
                  )
df.head()

,Revenue,EBITDA,Taxes,Capital Expenditure,Working Capital,Cash Flow,Discount Factor,Present Value of Free Cash Flow
Date,,,,,,,,
2022-12-31,0.18,0,0,0,0,0,0.943396,0
2023-12-31,0.18,0,0,0,0,0,0.943396,0
2024-12-31,0.18,0,0,0,0,0,0.943396,0
2025-12-31,0.18,0,0,0,0,0,0.943396,0
2026-12-31,0.18,0,0,0,0,0,0.943396,0


In [5]:
df["Revenue"] = (1 + df["Revenue"]).cumprod() * initial_revenue

df["EBITDA"] = df["Revenue"] * ebitda_margin

df["Taxes"] = -df["EBITDA"] * taxes

df["Capital Expenditure"] = -df["Revenue"] * capital_expenditures

df["Working Capital"] = -df["Revenue"] * working_capital_increase

df["Cash Flow"] = df["EBITDA"] + df["Taxes"] + df["Capital Expenditure"] + df["Working Capital"]

df["Discount Factor"] =  df["Discount Factor"].cumprod()

df["Present Value of Free Cash Flow"] = df["Cash Flow"] * df["Discount Factor"]


In [6]:
df

,Revenue,EBITDA,Taxes,Capital Expenditure,Working Capital,Cash Flow,Discount Factor,Present Value of Free Cash Flow
Date,,,,,,,,
2022-12-31,261.960000,117.882000,-24.755220,-7.858800,-15.717600,69.550380,0.943396,65.613566
2023-12-31,309.112800,139.100760,-29.211160,-9.273384,-18.546768,82.069448,0.889996,73.041517
2024-12-31,364.753104,164.138897,-34.469168,-10.942593,-21.885186,96.841949,0.839619,81.310368
2025-12-31,430.408663,193.683898,-40.673619,-12.912260,-25.824520,114.273500,0.792094,90.515315
2026-12-31,507.882222,228.547000,-47.994870,-15.236467,-30.472933,134.842730,0.747258,100.762332
2027-12-31,599.301022,269.685460,-56.633947,-17.979031,-35.958061,159.114421,0.704961,112.169388
2028-12-31,707.175206,318.228843,-66.828057,-21.215256,-42.430512,187.755017,0.665057,124.867810
2029-12-31,834.466743,375.510034,-78.857107,-25.034002,-50.068005,221.550920,0.627412,139.003788
2030-12-31,984.670757,443.101841,-93.051387,-29.540123,-59.080245,261.430086,0.591898,154.740066


In [7]:
present_value_of_projected_fcf = df["Present Value of Free Cash Flow"].drop(index="2032-12-31").sum()
present_value_of_projected_fcf

1114.2819602033007

In [8]:
terminal_value = (df.loc["2032", "EBITDA"] * ebitda_exit_multiple).values[0]
terminal_value

20175.082590019258

In [9]:
present_value_of_terminal_value = terminal_value * df.loc["2031" ,"Discount Factor"].values[0]
present_value_of_terminal_value

11265.660742097876

In [10]:
present_assets_value = present_value_of_projected_fcf + present_value_of_terminal_value
present_assets_value

12379.942702301178

In [12]:
message = """

Our Projected Present Value of Projected Future Cash Flows are: {0}.

We are providing revenue guidance in 2032 to be: {1}.

With an estimated EBITDA around: {2}.

The terminal value a this time given a multiple of {3}

and the above EBITDA for 2030, would equal: {4}.
""".format(present_value_of_projected_fcf,
           df.loc["2032", "Revenue"][0],
           df.loc["2032", "EBITDA"][0],
           32.70,
           terminal_value
           )
print(message)



Our Projected Present Value of Projected Future Cash Flows are: 1114.2819602033007.

We are providing revenue guidance in 2032 to be: 1371.0555616730721.

With an estimated EBITDA around: 616.9750027528825.

The terminal value a this time given a multiple of 32.7

and the above EBITDA for 2030, would equal: 20175.082590019258.

